# **Description (English):**
This is your frontend application, built with the Streamlit library, that users will interact with. It collects input data from users via various UI elements (e.g., select boxes, sliders), sends this data to your deployed FastAPI API, and then displays the predictions and relevant insights.

# **الوصف (العربية):**
هذا هو تطبيق الواجهة الأمامية الذي سيتفاعل معه المستخدمون، وهو مبني باستخدام مكتبة Streamlit. يقوم بجمع بيانات المدخلات من المستخدمين عبر عناصر واجهة المستخدم المختلفة (مثل القوائم المنسدلة، أشرطة التمرير)، ويرسل هذه البيانات إلى واجهة برمجة التطبيقات (API) التي قمت بنشرها، ثم يعرض التنبؤات والرؤى ذات الصلة.

In [ ]:
# -*- coding: utf-8 -*-
# streamlit_app.py

# --- 1. Import Libraries ---
# streamlit: The main library for building web applications.
# requests: Used to send HTTP requests to your FastAPI backend API.
# time: (Optional) Used for pausing execution, e.g., for a "loading" spinner effect.
import streamlit as st
import requests
import time

# --- 2. Basic Page Configuration ---
# Sets up the general appearance and properties of your Streamlit web page.
# Includes title, icon, layout (wide for more space), and sidebar initial state.
st.set_page_config(
    page_title="منصة توقع مغادرة العملاء",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- 3. Main User Interface (Frontend) ---
# This section defines the main content and layout of your Streamlit application.
# It includes titles, introductory text, input forms, and prediction display.

# Main title and subtitle of the application
st.title("💡 منصة توقع مغادرة العملاء")
st.markdown("### حلول متكاملة للاحتفاظ بعملائك وتحقيق النمو!")

# Introduction text
st.write("""
مرحبًا بك في منصة Churn Predictor! هنا، نحول بيانات عملائك إلى رؤى قوية لمساعدتك على **توقع العملاء المعرضين للمغادرة** قبل فوات الأوان.
نحن ندرك أن **الاحتفاظ بالعملاء أهم من اكتساب عملاء جدد**، وبرمجيتنا مصممة لهذا الغرض تحديدًا.
""")

st.markdown("---") # Horizontal separator line

# Layout with two columns: left for input form, right for marketing/educational content
col_left, col_right = st.columns([2, 1])

with col_left: # Content for the left column (Data Input Form)
    st.subheader("📊 أدخل تفاصيل العميل للتنبؤ بالمغادرة")

    # Input elements for customer data:
    # Each st.selectbox, st.slider, st.number_input collects a specific feature from the user.
    # IMPORTANT: Ensure the variable names and expected types match exactly with your CustomerData BaseModel in api_main.py.
    gender = st.selectbox("الجنس", ["Male", "Female"], help="ذكر أو أنثى.")
    senior_citizen = st.selectbox("هل هو عميل كبير في السن؟ (65+)", [0, 1], format_func=lambda x: "نعم" if x == 1 else "لا", help="اختر 1 إذا كان العميل فوق 65 عامًا، و 0 إذا لم يكن كذلك.")
    partner = st.selectbox("هل لديه شريك؟", ["Yes", "No"], help="هل العميل متزوج/لديه شريك؟")
    dependents = st.selectbox("هل لديه معالون؟", ["Yes", "No"], help="هل العميل لديه أطفال أو معالون آخرون؟")
    tenure = st.slider("مدة بقاء العميل (بالأشهر)", 0, 72, 12, help="كم شهرًا ظل العميل مشتركًا في الخدمة؟")
    phone_service = st.selectbox("هل لديه خدمة هاتف؟", ["Yes", "No"], help="هل العميل مشترك في خدمة الهاتف؟")
    multiple_lines = st.selectbox("هل لديه خطوط متعددة؟", ["Yes", "No", "No phone service"], help="إذا كان لديه خدمة هاتف، هل لديه خطوط متعددة؟")
    internet_service = st.selectbox("نوع خدمة الإنترنت", ["DSL", "Fiber optic", "No"], help="نوع خدمة الإنترنت المشترك بها العميل (أو لا يوجد).")
    online_security = st.selectbox("هل لديه أمان عبر الإنترنت؟", ["Yes", "No", "No internet service"], help="هل العميل مشترك في خدمة الأمان عبر الإنترنت؟")
    online_backup = st.selectbox("هل لديه نسخ احتياطي عبر الإنترنت؟", ["Yes", "No", "No internet service"], help="هل العميل مشترك في خدمة النسخ الاحتياطي عبر الإنترنت؟")
    device_protection = st.selectbox("هل لديه حماية للأجهزة؟", ["Yes", "No", "No internet service"], help="هل العميل مشترك في خدمة حماية الأجهزة؟")
    tech_support = st.selectbox("هل لديه دعم فني؟", ["Yes", "No", "No internet service"], help="هل العميل مشترك في خدمة الدعم الفني؟")
    streaming_tv = st.selectbox("هل لديه بث تلفزيوني؟", ["Yes", "No", "No internet service"], help="هل العميل مشترك في خدمة بث التلفزيون؟")
    streaming_movies = st.selectbox("هل لديه بث أفلام؟", ["Yes", "No", "No internet service"], help="هل العميل مشترك في خدمة بث الأفلام؟")
    contract = st.selectbox("نوع العقد", ["Month-to-month", "One year", "Two year"], help="نوع العقد الذي وقعه العميل (شهري، سنة، سنتين).")
    paperless_billing = st.selectbox("فاتورة لا ورقية؟", ["Yes", "No"], help="هل العميل مفعل خدمة الفاتورة الإلكترونية؟")
    payment_method = st.selectbox("طريقة الدفع", ["Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"], help="طريقة الدفع المفضلة للعميل.")
    monthly_charges = st.number_input("الرسوم الشهرية", min_value=0.0, max_value=200.0, value=50.0, help="إجمالي المبلغ الذي يدفعه العميل شهريًا.")
    total_charges = st.number_input("إجمالي الرسوم المدفوعة", min_value=0.0, max_value=8000.0, value=500.0, help="إجمالي المبلغ الذي دفعه العميل حتى الآن.")

    # Predict Button: This code block runs when the "Predict Churn" button is clicked.
    if st.button("توقع المغادرة"):
        # Prepare input data as a dictionary, matching the FastAPI CustomerData schema.
        input_data = {
            "gender": gender, "SeniorCitizen": senior_citizen, "Partner": partner,
            "Dependents": dependents, "tenure": tenure, "PhoneService": phone_service,
            "MultipleLines": multiple_lines, "InternetService": internet_service,
            "OnlineSecurity": online_security, "OnlineBackup": online_backup,
            "DeviceProtection": device_protection, "TechSupport": tech_support,
            "StreamingTV": streaming_tv, "StreamingMovies": streaming_movies,
            "Contract": contract, "PaperlessBilling": paperless_billing,
            "PaymentMethod": payment_method, "MonthlyCharges": monthly_charges,
            "TotalCharges": total_charges
        }

        # IMPORTANT: Replace this API URL with your actual deployed FastAPI URL (from Render or Ngrok).
        # This 'localhost' URL will NOT work when deployed!
        api_url = "http://localhost:8000/predict_churn/" # Placeholder! Change this URL!

        try:
            # Display a spinner while waiting for API response
            with st.spinner("جاري التنبؤ..."):
                response = requests.post(api_url, json=input_data, timeout=15) # Send POST request to API

            # Handle API response
            if response.status_code == 200:
                result = response.json()
                churn_prob = result.get("churn_probability", 0) # Get churn probability from response
                st.success(f"**احتمالية المغادرة المتوقعة: {churn_prob:.2f}**")

                # Display recommendations based on churn probability
                if churn_prob > 0.5: # Adjustable threshold for high churn risk
                    st.warning("⚠️ هذا العميل **عرضة للمغادرة!** يرجى اتخاذ إجراءات احترازية فورية.")
                    st.markdown("""
                    **توصيات فورية لاحتواء المغادرة:**
                    * **العملاء المعرضون للخطر (نقاط رئيسية):**
                        * **فترة البقاء القصيرة والرسوم العالية:** قدم لهم برامج ترحيب مميزة أو خصومات خاصة في الأشهر الأولى.
                        * **غياب الدعم الفني:** عروض على خدمة الدعم الفني، أو تواصل استباقي لحل المشكلات.
                        * **عقود شهرية:** حوافز للانتقال إلى عقود أطول (خصومات، خدمات إضافية).
                    * **الخطوة التالية:** تواصل مباشرة مع هؤلاء العملاء لفهم مشاكلهم وتقديم حلول مخصصة.
                    """)
                else:
                    st.info("✅ هذا العميل **من المرجح أن يبقى.** لا يزال من المهم مراقبة سلوكه.")
            else:
                st.error(f"❌ حدث خطأ أثناء التنبؤ: {response.status_code} - {response.text}")
                st.info("الرجاء التأكد من أن الـ API قيد التشغيل وأن الرابط صحيح (يجب أن يكون رابط Ngrok أو رابط خدمة النشر).")
        except requests.exceptions.ConnectionError:
            st.error("❌ لا يمكن الاتصال بخادم الـ API. تأكد من تشغيل الـ API بشكل صحيح وأن رابطه محدث.")
        except requests.exceptions.Timeout:
            st.error("❌ انتهت مهلة الاتصال بخادم الـ API. قد يكون الخادم مشغولاً أو الرابط غير مستقر. الرجاء المحاولة مرة أخرى.")
        except Exception as e:
            st.error(f"❌ حدث خطأ غير متوقع: {e}. الرجاء التحقق من المدخلات أو إعدادات الـ API.")

with col_right: # Content for the right column (Marketing & Educational Info)
    st.subheader("✨ لماذا هذه المنصة؟")
    st.markdown("""
    تُعد مغادرة العملاء تحديًا كبيرًا للشركات. تهدف منصتنا إلى تزويدك بـ **أداة قوية وذكية** تساعدك على:

    * **توقع العملاء المعرضين للمغادرة:** تعرف على من قد يغادر قبل أن يحدث ذلك.
    * **فهم الأسباب الكامنة:** لماذا يغادر العملاء؟ نقدم لك رؤى عميقة مبنية على الذكاء الاصطناعي (باستخدام **SHAP**) لفهم العوامل الرئيسية المؤثرة.
    * **صياغة استراتيجيات فعالة:** بناءً على التنبؤات والرؤى، يمكنك اتخاذ قرارات مستنيرة لوضع خطط احتفاظ بالعملاء ناجحة.
    """)

    st.markdown("---")

    st.subheader("💡 الفوائد التجارية الملموسة:")
    st.markdown("""
    * **زيادة الإيرادات:** تقليل المغادرة يعني زيادة في قاعدة العملاء الدائمة.
    * **توفير التكاليف:** تكلفة اكتساب عميل جديد أعلى بكثير من تكلفة الاحتفاظ بعميل حالي.
    * **تحسين رضا العملاء:** فهم مشاكل العملاء يؤدي إلى تحسين الخدمات والمنتجات.
    * **اتخاذ قرارات مبنية على البيانات:** بدلًا من التخمين، اعتمد على تحليل دقيق.
    """)

    st.markdown("---")

    st.subheader("🎁 جرب مجانًا!")
    st.markdown("""
    نقدم لك فرصة لتجربة منصتنا بأقل التزام:

    * **تجربة مجانية محدودة:** قم بتحميل مجموعة صغيرة من بيانات عملائك (مثلاً 100-500 سجل) للحصول على تقرير أولي مجانًا.
    * **حساب تجريبي (Demo Account):** استكشف لوحة المعلومات التفاعلية باستخدام بيانات وهمية لترى كيف تعمل المنصة وتقاريرها (مثل بيانات Telco Customer Churn التجريبية).
    * **دراسات حالة:** (سيتم إضافتها قريبًا) شاهد كيف ساعدت منصتنا شركات أخرى على تقليل المغادرة وتحقيق نتائج مذهلة.
    """)

    st.markdown("---")

    st.subheader("📞 الدعم والتدريب")
    st.markdown("""
    نحن هنا لمساعدتك في كل خطوة:
    * **الدعم الفني:** فريقنا متاح للإجابة على جميع استفساراتك حول تنسيق البيانات أو تفسير النتائج.
    * **المواد التعليمية:** نوفر فيديوهات قصيرة وأدلة استخدام مبسطة لمساعدتك على تحقيق أقصى استفادة من المنصة.
    """)

    st.markdown("---")

    st.subheader("💰 نماذج عملنا")
    st.markdown("""
    نقدم حلولاً مرنة تناسب احتياجات عملك:
    * **نموذج الاشتراك (SaaS):** اختر خطة شهرية أو سنوية بناءً على حجم بياناتك والميزات التي تحتاجها (عدد العملاء، المستخدمين، تقارير مخصصة، دمج مع CRM).
        * مثال: **الخطة الأساسية:** 50 دولارًا شهريًا لـ 5000 عميل. **الخطة المميزة:** 200 دولارًا شهريًا لـ 50,000 عميل + توصيات مخصصة.
    * **الدفع حسب الاستخدام:** ادفع فقط مقابل عدد التنبؤات التي تحتاجها.
    * **الخدمات الاستشارية:** نقدم تحليلات معمقة ومساعدة في تطبيق التوصيات ودمج الحلول مع أنظمتك.
    * **الشراكات:** نتعاون مع شركات استشارية ومقدمي حلول أعمال لتوسيع نطاق خدماتنا.
    """)

# Sidebar content (optional supplementary information)
st.sidebar.title("🚀 ابدأ الآن!")
st.sidebar.markdown("---")
st.sidebar.info("أدخل تفاصيل العميل في النموذج الرئيسي واحصل على تنبؤ فوري!")
st.sidebar.markdown("---")
st.sidebar.write("للاستفسارات أو الدعم، يرجى التواصل معنا.")
st.sidebar.markdown("[تواصل معنا](#)") # Placeholder link, replace with actual contact page